# Coursera Capstone Project

**Introduction/Business Problem:** We want to be able to find the best locations to start a restaurant in the Bay Area (the region encompassing San Francisco, CA and the cities and towns around it). We want to be able to attract the most customers and based on previous and nearby restaurants, we want to see if we can be successful in that area.